<a href="https://colab.research.google.com/github/BlackCurrantDS/Mining-Rare-Association-Rules/blob/master/FP_Processing_each_file_cancer_without_superset_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np
import csv
import fileinput

In [27]:
#for dictionary
originalfile = "/content/bcancer_data_original.csv"

df = pd.read_csv(originalfile)

#removing index column
cols = [0]
df.drop(df.columns[cols],axis=1,inplace=True)

for i,n in enumerate(df.columns):
      if n=='class':
        df[n] = 'class@' + df[n].astype(str)
      else:
        df[n] = 'A'+str(i)+'@' + df[n].astype(str)

uni = pd.DataFrame()

for col in df:
  uni= pd.concat([uni, pd.DataFrame(df[col].unique())], ignore_index=True)

my_dict = uni.to_dict()

my_dict = my_dict[0]


In [28]:
mapping = {
    
    'A0':'class'
    ,'A1':'AGE'
    ,'A2':'menopause'
    ,'A3':'tumor-size'
    ,'A4':'inv-nodes'
    ,'A5':'node-caps'
    ,'A6':'deg-malig'
    ,'A7':'breast'
    ,'A8':'breast-quad'
    ,'A9':'irradiat'
           
           }

In [29]:
#min_support = np.arange(0.01, .11, 0.01)
min_support = [.01]
d= "/content/"

In [30]:
def parse_input(filename):
                with open(filename) as f:
                    data = [set(literal_eval(line)) for line in f]
                return data

In [31]:
import os
from ast import literal_eval

for i in (min_support):
          print(round(i,2))
          print(str(round(i,2)).replace('0.', ''))
          file_to_process = d+"outputcancerRP"+str(round(i,2)).replace('0.', '')+".txt"
          superset_file = d+"superset"+str(round(i,2)).replace('0.', '')+".txt"
          superset_file_final = d+"superset"+str(round(i,2)).replace('0.', '')+"_final"+".txt"
          superset_file_final_v1 = d+"superset"+str(round(i,2)).replace('0.', '')+"_final_v1"+".txt"
          rule_file = d+"output_rules"+str(round(i,2)).replace('0.', '')+".txt"
          rule_file_final = d+"output_rules_final_"+str(round(i,2)).replace('0.', '')+".txt"
          
          f = open(file_to_process,'r')
          lst = []
          for line in f:
              line  = line.split("SUP", 1)
        
              line  = line[0]
              lst.append(line)
          f.close()
          f = open(file_to_process,'w')
          for line in lst:
              f.write(line+'\n')
          f.close()

          f = open(file_to_process,'r')
          lst = []
          for line in f:
              line  = line.split("#", 1)
        
              line  = line[0]
              lst.append(line)
          f.close()
          f = open(file_to_process,'w')
          for line in lst:
              f.write(line+'\n')
          f.close()

          rpp_process = pd.read_csv(file_to_process, header=None, names=["itemsets"],index_col=False)

          temp_df = pd.concat([rpp_process[['itemsets']], rpp_process['itemsets'].str.split(' ', expand=True)], axis=1)

          temp_df=temp_df.drop('itemsets', axis=1)

        #for class , moving the class column as first
          with open("/content/temp_file.csv", 'w') as f:
                for row in temp_df.itertuples(index=False):
                    #print(row)
                    ls = list(row)
                    ls = [x for x in ls if x]
                    try:
                      if '0' or '1' in ls:
                            old_index = ls.index('0') if '0' in ls else ls.index('1')
                            #print("old_index", old_index)
                            
                            ls.insert(0, ls.pop(old_index))
                            #print("after row", ls)
                            f.write(str(ls)+'\n')
                    except ValueError:
                      pass

          def compute_output(output_file, data, filter_value):
              ls = data
              index_to_pop=[]
              for set1 in ls:
                #print("set1", set1)
                for set2 in ls:
                    if set1 is set2:
                        # Do not try to compare a row with itself
                        continue
                    elif 1==2:
                          if set1 in ls:
                            index = ls.index(set1)
                            index_to_pop.append(index)

                            break
              print("Final index_to_pop",index_to_pop)              
              for index in sorted(index_to_pop, reverse=True):
                  del ls[index]           
              print("Final list",ls)

              f = open(superset_file,'w')
              for line in ls:
                f.write(str(line)+'\n')
              f.close()

          def filter_file(path, filter_value=3, in_name='temp_file.csv', out_name='filteredSets'):
              data = parse_input(os.path.join(path, in_name))
              print("data", data)
              output_filename = os.path.join(path, '{}{}'.format(out_name, filter_value))
              with open(output_filename, 'w') as out_file:
                  compute_output(out_file, data, filter_value)

          filter_file('/content')

          print("longest ones sorted")

          with open(superset_file) as filein, open(superset_file_final,'w') as fileout:
                    for line in filein:
                            line=line.replace("'","")
                            line=line.replace("}","")
                            line=line.replace("{","")
                            line=line.replace(", "," ")
                            fileout.write(line)

            
          with open(superset_file_final) as filein, open(superset_file_final_v1,'w') as fileout:
              for line in filein:
                  line=line.replace("[","")
                  line=line.replace("]","")
                  fileout.write(line)

          rpp_process = pd.read_csv(superset_file_final_v1, header=None, names=["itemsets"],index_col=False)

          temp_df = pd.concat([rpp_process[['itemsets']], rpp_process['itemsets'].str.split(' ', expand=True)], axis=1)

          temp_df=temp_df.drop('itemsets', axis=1)

          with open(superset_file, 'w') as f:
                  for row in temp_df.itertuples(index=False):
                      ls = list(row)
                      ls = [x for x in ls if x]
                      try:
                        if '0' or '1' in ls:
                              old_index = ls.index('0') if '0' in ls else ls.index('1')
                              #print("old_index", old_index)
                              
                              ls.insert(0, ls.pop(old_index))
                              #print("after row", ls)
                              f.write(str(ls)+'\n')
                      except ValueError:
                        pass

          with open(superset_file) as filein, open(superset_file_final,'w') as fileout:
                    for line in filein:
                            line=line.replace("'","")
                            line=line.replace("}","")
                            line=line.replace("{","")
                            line=line.replace(", "," ")
                            fileout.write(line)

            
          with open(superset_file_final) as filein, open(superset_file_final_v1,'w') as fileout:
              for line in filein:
                  line=line.replace("[","")
                  line=line.replace("]","")
                  fileout.write(line)

          superset = pd.read_csv(superset_file_final_v1,header=None, names=["itemsets"],index_col=False)

          superset_df = pd.concat([superset[['itemsets']], superset['itemsets'].str.split(' ', expand=True)], axis=1
                  )
            
          superset_df.to_csv("superset_df.csv", header=None, index=False)

          print("Superset is done")

            
            #superset_df = superset_df.loc[superset_df.iloc[:,1].isin(['1','0'])] #filtering rows

          #superset_df = superset_df.loc[superset_df.iloc[:,1].isin(['1'])] #filtering only for minority

          superset_df.to_csv("superset_df_1.csv", header=None, index=False)

          superset_df=superset_df.drop('itemsets', axis=1)

          print("lenght of columns", len(superset_df.columns))
          if len(superset_df.columns) ==10:
              superset_df.columns = ['a', 'b', 'c', 'd', 'e', 'f','g', 'h', 'i', 'j']
          
          elif len(superset_df.columns) ==9:
              superset_df.columns = ['a', 'b', 'c', 'd', 'e', 'f','g', 'h', 'i']
          elif len(superset_df.columns) ==8:
              superset_df.columns = ['a', 'b', 'c', 'd', 'e', 'f','g', 'h']

          elif len(superset_df.columns) ==7:
              superset_df.columns = ['a', 'b', 'c', 'd', 'e', 'f','g']

          elif len(superset_df.columns) ==6:
              superset_df.columns = ['a', 'b', 'c', 'd', 'e', 'f']

          elif len(superset_df.columns) ==5:
              superset_df.columns = ['a', 'b', 'c', 'd', 'e']
          elif len(superset_df.columns) ==4:
              superset_df.columns = ['a', 'b', 'c', 'd']

          for i,n in enumerate(superset_df.columns):
  
                    superset_df[n] = '@' + superset_df[n].astype(str)

          di = {f'@{k}': v for k, v in my_dict.items()}

            
          for col in superset_df:
              superset_df[col] = superset_df[col].replace(di)

          print("Mapping is done")
          superset_df.to_csv("superset_df_1_after_mapping.csv", header=None, index=False) 
          
          if len(superset_df.columns) ==10:

              superset_df['rules'] = superset_df['b']+','+superset_df['c']+','+superset_df['d']+','+superset_df['e']+','+superset_df['f']+','+superset_df['g']+','+superset_df['h']+','+superset_df['i']+','+superset_df['j']
          
          elif len(superset_df.columns) ==9:

              superset_df['rules'] = superset_df['b']+','+superset_df['c']+','+superset_df['d']+','+superset_df['e']+','+superset_df['f']+','+superset_df['g']+','+superset_df['h']+','+superset_df['i']
          elif len(superset_df.columns) ==8:
              superset_df['rules'] = superset_df['b']+','+superset_df['c']+','+superset_df['d']+','+superset_df['e']+','+superset_df['f']+','+superset_df['g']+','+superset_df['h']
          
          elif len(superset_df.columns) ==7:
              superset_df['rules'] = superset_df['b']+','+superset_df['c']+','+superset_df['d']+','+superset_df['e']+','+superset_df['f']+','+superset_df['g']
          
          elif len(superset_df.columns) ==6:
              superset_df['rules'] = superset_df['b']+','+superset_df['c']+','+superset_df['d']+','+superset_df['e']+','+superset_df['f']
          
          elif len(superset_df.columns) ==5:
              superset_df['rules'] = superset_df['b']+','+superset_df['c']+','+superset_df['d']+','+superset_df['e']
          
          elif len(superset_df.columns) ==4:
              superset_df['rules'] = superset_df['b']+','+superset_df['c']+','+superset_df['d']
          
          
          superset_df['pos'] = superset_df['rules'].str.find('@None')
          superset_df['rules'] = superset_df.apply(lambda x: x['rules'][0:x['pos']],axis=1)
          superset_df['rules'] = superset_df['rules']+">"+superset_df['a']
          superset_df['rules'] = superset_df['rules'].str.replace(',@,','')
          superset_df['rules'] = superset_df['rules'].str.replace(',>','>')

          superset_df['rules'].to_csv(rule_file_final, header=None, index=False)
          #for classifer file
          superset_df['rules'].to_csv(rule_file, header=None, index=False)

          print("Rules generated")
          text = rule_file_final
          fields = mapping

          lst = []
          for line in fileinput.input(text):
              for field in fields:
                  if field in line:
                      line = line.replace(field, fields[field])
              lst.append(line)

          print(lst)

          f = open(rule_file_final,'w')
          for line in lst:
              f.write(line)
          f.close()

          print("Rules mapping done")

0.01
01
data [{'30', '1', '37', '35', '44', '32', '9', '4', '12', '28'}, {'22', '11', '1', '5', '32', '9', '42', '35', '29', '43'}, {'22', '34', '1', '36', '44', '21', '9', '4', '38', '29'}, {'27', '30', '1', '44', '32', '3', '9', '35', '40', '15'}, {'27', '30', '1', '44', '32', '3', '35'}, {'27', '8', '11', '30', '1', '44', '32', '3', '35', '39'}, {'6', '34', '1', '37', '44', '13', '9', '24', '31', '35'}, {'8', '30', '1', '5', '36', '19', '32', '38', '26', '43'}, {'8', '30', '1', '37', '35', '32', '13', '3', '26', '43'}, {'8', '11', '30', '1', '37', '35', '32', '3', '26', '43'}, {'8', '30', '1', '32', '26', '43'}, {'8', '30', '1', '37', '35', '32', '3', '26', '43'}, {'22', '34', '1', '5', '12', '31', '10', '35', '43'}, {'22', '34', '1', '5', '12', '31', '10', '35', '39', '43'}, {'22', '34', '1', '37', '5', '12', '31', '10', '35', '43'}, {'8', '22', '14', '2', '33', '1', '36', '41', '29', '43'}, {'8', '30', '1', '5', '36', '24', '43', '39', '16', '33'}, {'22', '1', '36', '29', '32', '9